The purpose of this noteboook is to update the `metabolic_reactions.tsv` flat file in ecoli reconstruction with a new column indicating whether the reactions are new. i.e. added in 2022. 

In [1]:
import numpy as np
import ast
import seaborn as sns
import pandas as pd
import os
import matplotlib.pyplot as plt
import dill
import requests
import xmltodict
import cvxpy as cp
import itertools
from scipy.special import logsumexp
%matplotlib inline
# sns.set(style='darkgrid', palette='viridis', context='talk')

os.chdir(os.path.expanduser('~/dev/vivarium-ecoli/'))


In [2]:
FLAT_DIR = f'reconstruction/ecoli/flat/'
NOTEBOOK_DIR = f'notebooks/Heena notebooks/'

In [3]:
# create a function to deal with non-homogeous dtype of columns. mixture of strings and lists
def string_to_list(s):
    try:
        return np.array(ast.literal_eval(s))  # Safely convert string to array
    except (ValueError, SyntaxError):
        return np.array(s)  # Return as array if object is not a list and conversion fails (e.g., NaN values)

In [4]:
# load annotated new metabolic gene tsv and metabolic_reactions.tsv
metabolic_gene_annotation = pd.read_csv(os.path.join(NOTEBOOK_DIR, "metabolic_gene_annotation.csv"), converters={'Reactions': string_to_list})
metabolic_reactions = pd.read_csv(os.path.join(FLAT_DIR, "metabolic_reactions.tsv"),sep='\t', skiprows=4)
modified = pd.read_csv(os.path.join(FLAT_DIR, "metabolic_reactions_modified.tsv"),sep='\t')
added = pd.read_csv(os.path.join(FLAT_DIR, "metabolic_reactions_added.tsv"),sep='\t', skiprows=1)

In [5]:
# add new column in metabolic_reactions indicating whether it is a new reactions
is_new = [] # whether a metabolic rxn is new or not
rxn_in_metabolic = [] #new rxn names that are in metabolic rxn
all_new_rxn = np.hstack(metabolic_gene_annotation['Reactions'])
all_new_rxn = [rxn for rxn in all_new_rxn if rxn != ''] #remove empty rxns
all_new_rxn = np.unique(all_new_rxn) #remove duplicates

for idx,reaction in enumerate(metabolic_reactions.iloc[:,0]):
    is_new_reaction_described_in_reactions = [new_rxn in reaction for new_rxn in all_new_rxn]
    
    if any(is_new_reaction_described_in_reactions): #if reaction name in metabolic_reactions is found in all_new_rxn
        # a reaction may have several variations catalyzed by different enzymes, which leads to longer reaction name stored in sim_data
        # only the 'new_reactions' catalyzed by that encoded in metabolic_gene_annotation['Enzymes'] are the correct new_reactions

        # get row index of new reaction in metabolic_gene_annotation - get enzyme encoded by new gene catalyzing the reaction
        new_rxn_name = all_new_rxn[is_new_reaction_described_in_reactions]
        index_new_rxn = metabolic_gene_annotation[metabolic_gene_annotation['Reactions'].apply(
                        lambda x: any(rxn in x for rxn in new_rxn_name))].index
        enzyme_encoded_new = metabolic_gene_annotation["Enzyme encoded"][index_new_rxn]
        enzyme_encoded_reaction = metabolic_reactions.catalyzed_by[idx] #catalyzed_by of metabolic_reactions of the reaction
        
        rxn_in_metabolic.append(new_rxn_name)
        if any(enzyme in enzyme_encoded_reaction for enzyme in enzyme_encoded_new):
            is_new.append(1) #True
        else:
            is_new.append(0) #False
    else:
        is_new.append(0) #False

metabolic_reactions["is_new"] = is_new

In [6]:
# upon initial investigation of the is_new matrix, I realize that not all new rxns are included in the metabolic_reactions.tsv file. 
# I want to extract those new rxns that aren't included and manually investigate them to determine whether they shuld be included or not
rxn_in_metabolic = np.unique(np.hstack(rxn_in_metabolic))
print(f'There are {len(all_new_rxn) - len(rxn_in_metabolic)} new reactions missing from the metabolic_reactions.tsv file:')
print(all_new_rxn[~np.isin(all_new_rxn, rxn_in_metabolic)])


There are 15 new reactions missing from the metabolic_reactions.tsv file:
['1.5.1.34-RXN' '3-NUCLEOTID-RXN' 'CYCPHOSDIESTER-RXN' 'R303-RXN'
 'RXN-15119' 'RXN-16650' 'RXN-24038' 'RXN-24042' 'RXN-24048' 'RXN-24049'
 'RXN-24479' 'RXN0-1804' 'RXN0-5001' 'RXN0-5285' 'TRANS-RXN-387']


In [7]:
all_new_rxn[~np.isin(all_new_rxn, rxn_in_metabolic)]

array(['1.5.1.34-RXN', '3-NUCLEOTID-RXN', 'CYCPHOSDIESTER-RXN',
       'R303-RXN', 'RXN-15119', 'RXN-16650', 'RXN-24038', 'RXN-24042',
       'RXN-24048', 'RXN-24049', 'RXN-24479', 'RXN0-1804', 'RXN0-5001',
       'RXN0-5285', 'TRANS-RXN-387'], dtype='<U33')

In [8]:
len(rxn_in_metabolic)

350

In [9]:
metabolic_reactions.head(30)

,id,stoichiometry,direction,catalyzed_by,is_new
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-RXN,"{""ACYL-SN-GLYCEROL-3P[CCO-CYTOSOL]"": -1, ""ACYL...",L2R,"[""1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER""]",0
1,1.1.1.127-RXN,"{""2-DEHYDRO-3-DEOXY-D-GLUCONATE[CCO-CYTOSOL]"":...",BOTH,"[""KDUD-MONOMER""]",0
2,1.1.1.215-RXN,"{""GLUCONATE[CCO-CYTOSOL]"": -1, ""NADP[CCO-CYTOS...",R2L,"[""CPLX0-235""]",0
3,1.1.1.251-RXN,"{""GALACTITOL-1-PHOSPHATE[CCO-CYTOSOL]"": -1, ""N...",BOTH,"[""CPLX0-8186""]",0
4,1.1.1.271-RXN,"{""CPD-13118[CCO-CYTOSOL]"": -1, ""NADP[CCO-CYTOS...",R2L,"[""CPLX0-3881""]",1
5,1.1.1.274-RXN,"{""CPD-377[CCO-CYTOSOL]"": -1, ""NADP[CCO-CYTOSOL...",R2L,[],0
6,1.1.1.283-RXN,"{""LACTALD[CCO-CYTOSOL]"": -1, ""NADP[CCO-CYTOSOL...",R2L,"[""METHYLGLYREDUCT-MONOMER""]",0
7,1.1.1.39-RXN,"{""MAL[CCO-CYTOSOL]"": -1, ""NAD[CCO-CYTOSOL]"": -...",L2R,"[""MALIC-NAD-CPLX""]",0
8,1.1.1.83-RXN,"{""CPD-660[CCO-CYTOSOL]"": -1, ""NAD[CCO-CYTOSOL]...",L2R,"[""G6986-MONOMER""]",0
9,1.11.1.15-RXN,"{""Red-Thioredoxin[CCO-CYTOSOL]"": -1, ""Alkyl-Hy...",L2R,"[""CPLX0-7747"", ""EG10108-MONOMER"", ""CPLX0-8157""]",1


In [10]:
# Update supplementary metabolic reactions files - modifed and added
# all added reactions aren't new. They are reactions not included in ecocyc and are manually added
added.insert(loc=4,column='is_new',value=0)

# all modified reactions happen to be old. Make sense though because we probably didn't try modifing any of the new reactions
modified.insert(loc=4,column='is_new',value=0)

In [11]:
# save updated metabolic reactions as tsv
col_list = ["id", "direction"]
for colname in col_list:
    metabolic_reactions[colname] = metabolic_reactions[colname].apply(lambda x:'"' + str(x) + '"')
    added[colname] = added[colname].apply(lambda x:'"' + str(x) + '"')
    modified[colname] = modified[colname].apply(lambda x:'"' + str(x) + '"')

added["_comments"] = added["_comments"].apply(lambda x:'"' + str(x) + '"')
modified["_comments"] = modified["_comments"].apply(lambda x:'"' + str(x) + '"')

In [12]:
metabolic_reactions.head(2)

,id,stoichiometry,direction,catalyzed_by,is_new
0,"""1-ACYLGLYCEROL-3-P-ACYLTRANSFER-RXN""","{""ACYL-SN-GLYCEROL-3P[CCO-CYTOSOL]"": -1, ""ACYL...","""L2R""","[""1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER""]",0
1,"""1.1.1.127-RXN""","{""2-DEHYDRO-3-DEOXY-D-GLUCONATE[CCO-CYTOSOL]"":...","""BOTH""","[""KDUD-MONOMER""]",0


In [13]:
modified.head() 

,id,stoichiometry,direction,catalyzed_by,is_new,_comments
0,"""ADENOSINE-NUCLEOSIDASE-RXN""","{""ADENOSINE[CCO-CYTOSOL]"": -1, ""WATER[CCO-CYTO...","""L2R""","[""EG11082-MONOMER""]",0,"""Replaced D-Ribofuranose (compound class that ..."
1,"""INOSINE-NUCLEOSIDASE-RXN""","{""INOSINE[CCO-CYTOSOL]"": -1, ""WATER[CCO-CYTOSO...","""L2R""","[""EG11082-MONOMER""]",0,"""Replaced D-Ribofuranose (compound class that ..."
2,"""RXN-14882[CCO-CYTOSOL]-CPD-15818//D-Ribofuran...","{""CPD-15818[CCO-CYTOSOL]"": -1, ""CPD0-1108[CCO-...","""BOTH""",[],0,"""Replaced D-Ribofuranose (compound class that ..."
3,"""RXN-14882[CCO-EXTRACELLULAR]-CPD-15818//D-Rib...","{""CPD-15818[CCO-EXTRACELLULAR]"": -1, ""CPD0-110...","""BOTH""",[],0,"""Replaced D-Ribofuranose (compound class that ..."
4,"""RXN-14882[CCO-PERI-BAC]-CPD-15818//D-Ribofura...","{""CPD-15818[CCO-PERI-BAC]"": -1, ""CPD0-1108[CCO...","""BOTH""",[],0,"""Replaced D-Ribofuranose (compound class that ..."


In [14]:
import csv
metabolic_reactions[:0].to_csv("reconstruction/ecoli/flat/metabolic_reactions.tsv", sep = "\t", index=False, quoting=csv.QUOTE_NONNUMERIC)
metabolic_reactions.to_csv("reconstruction/ecoli/flat/metabolic_reactions.tsv", sep = "\t", index = False, header = False, mode="a", quoting=csv.QUOTE_NONE)

In [17]:
added[:0].to_csv("reconstruction/ecoli/flat/metabolic_reactions_added.tsv", sep = "\t", index=False, quoting=csv.QUOTE_NONNUMERIC)
modified[:0].to_csv("reconstruction/ecoli/flat/metabolic_reactions_modified.tsv", sep = "\t", index=False, quoting=csv.QUOTE_NONNUMERIC)

added.to_csv("reconstruction/ecoli/flat/metabolic_reactions_added.tsv", sep = "\t", index = False, header = False, mode="a", quoting=csv.QUOTE_NONE)
modified.to_csv("reconstruction/ecoli/flat/metabolic_reactions_modified.tsv", sep = "\t", index = False, header = False, mode="a", quoting=csv.QUOTE_NONE)

# Scratch

In [39]:
sum(is_new)

530

In [38]:
boo = (metabolic_reactions['direction'] == '"R2L"') & (metabolic_reactions['is_new'] == 1)
# boo = (metabolic_reactions['direction'] == '"R2L"')
metabolic_reactions[boo]

,id,stoichiometry,direction,catalyzed_by,is_new
4,"""1.1.1.271-RXN""","{""CPD-13118[CCO-CYTOSOL]"": -1, ""NADP[CCO-CYTOS...","""R2L""","[""CPLX0-3881""]",1
720,"""CROBETREDUCT-RXN""","{""GAMMA-BUTYROBETAINYL-COA[CCO-CYTOSOL]"": -1, ...","""R2L""","[""CROBETREDUCT-MONOMER""]",1
781,"""DIENOYLCOAREDUCT-RXN""","{""T2-DECENOYL-COA[CCO-CYTOSOL]"": -1, ""NADP[CCO...","""R2L""","[""DIENOYLCOAREDUCT-MONOMER""]",1
864,"""FCLREDUCT-RXN""","{""CPD-13118[CCO-CYTOSOL]"": -1, ""NADP[CCO-CYTOS...","""R2L""","[""CPLX0-3881""]",1
1031,"""HYDROXYLAMINE-REDUCTASE-RXN""","{""AMMONIUM[CCO-CYTOSOL]"": -1, ""Acceptor[CCO-CY...","""R2L""","[""G6457-MONOMER""]",1
2110,"""RXN-19619""","{""TRIMETHYLAMINE[CCO-PERI-BAC]"": -1, ""an-oxidi...","""R2L""","[""G7022-MONOMER""]",1
2171,"""RXN-20148-CPD-9956/OXYGEN-MOLECULE//SUPER-OXI...","{""CPD-9956[CCO-PM-BAC-NEG]"": -1, ""OXYGEN-MOLEC...","""R2L""","[""CYTOCHROME-B561-MONOMER""]",1
2178,"""RXN-20161-Ox-Thioredoxin/a-reduced-DsbD-prote...","{""Ox-Thioredoxin[CCO-CYTOSOL]"": -1, ""a-reduced...","""R2L""","[""RED-THIOREDOXIN-MONOMER""]",1
2179,"""RXN-20161-Ox-thioredoxins-1/a-reduced-DsbD-pr...","{""Ox-thioredoxins-1[CCO-CYTOSOL]"": -1, ""a-redu...","""R2L""","[""RED-THIOREDOXIN-MONOMER""]",1
2180,"""RXN-20161-Oxidized-CcmG-Proteins/a-reduced-Ds...","{""Oxidized-CcmG-Proteins[CCO-CYTOSOL]"": -1, ""a...","""R2L""","[""RED-THIOREDOXIN-MONOMER""]",1


In [37]:
np.unique(metabolic_reactions['direction'])

array(['"BOTH"', '"L2R"', '"R2L"'], dtype=object)